### Failure analysis
In this notebook, we analyze failure cases of the model (examples where model consistently fails), and try to make the model learn by giving similar examples in the prompt.
We will use GPT-3.5, no chain-of-thought.

In [3]:
import evaluation

In [4]:
ev1 = evaluation.Evaluation.from_config("config.json", "gpt-4")
ev1.run_example("remove", k=1, n=2)

Running example: remove ; attempt: 1 ; error depth: 1
Generated program from GPT:
def remove(head: Node, val: int) -> Optional[Node]:
    """Removes the first node with the given value from the list."""
    Requires(is_list(head))
    Ensures(Implies(Result() is not None, is_list(Result())))
    Unfold(is_list(head))
    if head.val == val:
        return head.next
    if head.next is None:
        Fold(is_list(head))
        return head
    head.next = remove(head.next, val)
    Fold(is_list(head))
    return head
response ['', 'Verification successful', 'Verification took 2.10 seconds.']
Verification result:
 Verification successful 




(True, (1, 1))

A pattern of error is Unfold before Unfolding the same node. We craft a minimal example that demonstrates this error.
```python
def example_err_1(head: Node) -> str:
    "Unfold before Unfolding the same node"
    Requires(is_list(head))
    Unfold(is_list(head))
    if Unfolding(is_list(head), head.next) is None:
        return "Singleton"
    return "Multiple"
```
The fix is simply to remove the Unfold statement. We added this example to `config.json` and updatd the verification error pickle (by running the script in scripts.ipynb). Now we analyze if this resolves the error.

In [ ]:
ev2 = evaluation.Evaluation.from_config("config_with_sorting.json", "gpt-4")
ev2.run_example("insertion_sort", k=3, n=3)

In [ ]:
ev3 = evaluation.Evaluation.from_config("config.json", "gpt-3.5-turbo-1106")
ev3.run_eval(k=3, n=3)

In [ ]:
few_shot_prompt = ev1.model.get_prompt("merge")
program_snippet = (
    "Placeholder for code produced by gpt"  # model.get_response(few_shot_prompt)
)
verif_result = "Verification failed: blah blah blah"  # nagini.verify(program_snippet)
ev1.model.extend_prompt(few_shot_prompt, program_snippet, verif_result)

1. We send few shot prompt to the model
1a. few_shot_prompt = ev1.model.get_prompt("merge")
1b. program_snippet = model.get_response(few_shot_prompt)
2. We verify the program snippet produced by the model
verif_result =  nagini.verify(program_snippet)

3. If there's an error, we want the ability to add an example that demonstrates the mistake to the prompt.
First, we let the user create an example. Let the example be now saved to `example_uuid_unverified.txt`.
Now we run nagini on the example to get the verification error. 
Then the user provides a verified example. We run nagini just to sanity check that verification succeeds. Let the verified example be saved to `example_uuid_verified.txt`. <hr>
Now, we extend the prompt with {unverified_example+error, verified_example}

4. We further extend the prompt with the program snippet (i.e. output from gpt) and the verification result from step 2.
few_shot_prompt = ev1.model.extend_prompt(few_shot_prompt, program_snippet, verif_result)
Go to 1b. until verification succeeds.


Requirement engineering for 3.
How do we get the user to provide an example that demonstrates the error?
We can use the following strategy:
1. We show the user the program snippet and the verification error.
2. We create a file `example_uuid_unverified.txt` and ask the user to provide an example that demonstrates the error. The user edits the file and signals us when done. We verify the example provided by the user.
How does the user signal us that he has saved the file?


3. We ask the user to provide a verified example.


In [ ]:
ev1 = evaluation.Evaluation.from_config("config.json", "gpt-3.5-turbo-1106")
ev1.data = ev1.data.clone("test-1")

In [ ]:
import interactive_widget

widget = interactive_widget.PromptAndVerifyWidget(ev1, "append")
display(widget.widget)

In [ ]:
widget.run_interactive()

### Test gpt-3.5-1106 vs gpt-3.5-0613

In [1]:
import evaluation
ev_1106 = evaluation.Evaluation.from_config("config.json", "gpt-3.5-turbo-1106")
ev_1106.run_eval(k=1, n=3)

Running example: prepend ; attempt: 1 ; error depth: 1
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))
    n = Node(val, head)
    return n
response ['', 'Verification failed', 'Errors:', 'Postcondition of prepend might not hold. There might be insufficient permission to access is_list(Result()). (tmp.py@24.12)', 'Verification took 6.27 seconds.']
Verification result:
 Verification failed: Postcondition of prepend might not hold. There might be insufficient permission to access is_list(Result()). at line 4.12 


Running example: prepend ; attempt: 1 ; error depth: 2
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Ensures(is_list(Result()))
    n = Node(val, head)
    return n
response ['', 'Verification failed', 'Errors:', 'Postcondition of prepend migh

EvalResult(results={'prepend': False, 'append': False, 'find': False, 'find_iter': False, 'remove': False, 'join_lists': True, 'reverse': False}, verified_at={'join_lists': (1, 2)})

In [2]:
ev_0613 = evaluation.Evaluation.from_config("config.json", "gpt-3.5-turbo-0613")
ev_0613.run_eval(k=1, n=3)

Running example: prepend ; attempt: 1 ; error depth: 1
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))
    n = Node(val, head)
    Fold(is_list(head))
    return n
response ['', 'Verification failed', 'Errors:', 'Fold might fail. There might be insufficient permission to access head.val. (tmp.py@26.4)', 'Verification took 2.47 seconds.']
Verification result:
 Verification failed: Fold might fail. There might be insufficient permission to access head.val. at line 6.4 


Running example: prepend ; attempt: 1 ; error depth: 2
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))
    n = Node(val, head)
    Fold(is_list(head))
    Fold(is_list(n))
    return n
response ['', 'Verification failed', 'Errors:', '

EvalResult(results={'prepend': True, 'append': False, 'find': False, 'find_iter': False, 'remove': False, 'join_lists': True, 'reverse': False}, verified_at={'prepend': (1, 3), 'join_lists': (1, 1)})

### is gpt-4 better at learning from errors?
test with append and depth=3

In [4]:
ev_gpt4 = evaluation.Evaluation.from_config("config.json", "gpt-4")
ev_gpt4.run_eval(k=1, n=3)

Running example: prepend ; attempt: 1 ; error depth: 1
Generated program from GPT:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Requires(is_list(head))
    Ensures(is_list(Result()))
    n = Node(val, head)
    Fold(is_list(n))
    return n
response ['', 'Verification successful', 'Verification took 3.15 seconds.']
Verification result:
 Verification successful 


Running example: append ; attempt: 1 ; error depth: 1
Generated program from GPT:
def append(head: Node, val: int) -> None:
    """Appends a new node with the given value to the end of the list."""
    Requires(is_list(head))
    Ensures(is_list(head))
    
    Unfold(is_list(head))
    if head.next is None:
        n = Node(val)
        head.next = n
        Fold(is_list(head))
    else:
        append(head.next, val)
        Fold(is_list(head))
response ['', 'Verification failed', 'Errors:', 'Fold might fail. There might be insufficient permission to access is

EvalResult(results={'prepend': True, 'append': True, 'find': False, 'find_iter': False, 'remove': False, 'join_lists': True, 'reverse': False}, verified_at={'prepend': (1, 1), 'append': (1, 2), 'join_lists': (1, 1)})

### Add postcondition to `find` as it is a bit awkward

In [6]:
import data
dataset = data.Data("config.json")
dataset = dataset.clone("find_with_post")

try gpt-3.5 on this dataset for `find`
gpt-3.5 still replaced the postcondition with `Ensures(Implies(Result() is not None, is_list(Result())))`
How about new gpt-3.5? same.
How about gpt-4?


In [3]:
ev_0613_find = evaluation.Evaluation.from_config("config.json", "gpt-4")
ev_0613_find.run_example("find", k=3, n=3)

Running example: find ; attempt: 1 ; error depth: 1
Generated program from GPT:
def find(head: Optional[Node], val: int) -> Optional[Node]:
    """Returns the first node with the given value, or None if not found."""
    Requires(Implies(head is not None, is_list(head)))
    Ensures(Implies(Result() is not None, is_list(Result())))
    if head is None:
        return None
    Unfold(is_list(head))
    if head.val == val:
        Fold(is_list(head))
        return head
    result = find(head.next, val)
    Fold(is_list(head))
    return result
response ['', 'Verification failed', 'Errors:', 'Fold might fail. There might be insufficient permission to access is_list(head.next). (tmp.py@32.4)', 'Verification took 2.08 seconds.']
Verification result:
 Verification failed: Fold might fail. There might be insufficient permission to access is_list(head.next). at line 12.4 


Running example: find ; attempt: 1 ; error depth: 2
Generated program from GPT:
def find(head: Optional[Node], val: int)

(False, None)

In [14]:
print(ev_0613_find.model.get_prompt("find")[-1]["content"])

def find(head: Optional[Node], val: int) -> Optional[Node]:
    """Returns the first node with the given value, or None if not found."""
    Ensures(Implies(head is not None, is_list(head)))
    if head is None:
        return None
    if head.val == val:
        return head
    result = find(head.next, val)
    return result

Verification failed: Conditional statement might fail. There might be insufficient permission to access head.val. at line 5.7


In [20]:
data_remove_with_eg = data.Data("config.json", "nagini_examples/test_remove_1-3.5_1106")
ev_gpt_4_remove = evaluation.Evaluation(data_remove_with_eg, "gpt-4")
ev_gpt_4_remove.run_example("remove", n=6)

Running example: remove ; attempt: 1 ; error depth: 1
Generated program from GPT:
def remove(head: Node, val: int) -> Optional[Node]:
    """Removes the first node with the given value from the list."""
    Requires(is_list(head))
    Ensures(Implies(Result() is not None, is_list(Result())))
    Unfold(is_list(head))
    if head.val == val:
        return head.next
    if head.next is None:
        Fold(is_list(head))
        return head
    head.next = remove(head.next, val)
    Fold(is_list(head))
    return head
response ['', 'Verification successful', 'Verification took 1.90 seconds.']
Verification result:
 Verification successful 




(True, (1, 1))

In [21]:
ev_gpt_4_remove.model.get_prompt("remove")

[{'role': 'system',
  'content': 'You are an assistant that given a python program, \nannotates it with appropriate Nagini annotations so that verification succeeds.\n\nNagini is a static verifier for Python. \nOur aim is to given a statically typed Python program, \nto come up with appropriate preconditions (e.g. Requires(is_list(head))), postcondition (e.g. Ensures(is_list(Result()))),\nloop invariants (Invariant(<assertion>)), predicate fold/unfolds (e.g. Fold(is_list(head)) / Unfold(is_list(head))) \nso that the program verifies correctly. Unfolding(e1, e2) evaluates e2 in the context where predicate e1 is temporarily unfolded.\n\nThe user will provide Python code and the verification errors. \nYou must add or change the specifications so that the resulting code verifies correctly. \nReturn only the code without any explanation or wrapping.\n\nAt the end, the user may provide multiple {program, verification errors} for the same code.\nYou must learn from the errors and return a cor